<img src="https://www.th-koeln.de/img/logo.svg" style="float:right;" width="200">

# 10th exercise: <font color="#C70039">Interpretable Machine Learning by means of Shapley Values</font>
* Course: AML
* Lecturer: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Author of notebook: <a href="https://www.gernotheisenberg.de/">Gernot Heisenberg</a>
* Date:   04.08.2025

---------------------------------

<img src="https://shap.readthedocs.io/en/latest/_images/example_notebooks_overviews_An_introduction_to_explainable_AI_with_Shapley_values_13_0.png" style="float: center;" width="600">

---------------------------------
**GENERAL NOTE 1**: 
Please make sure you are reading the entire notebook, since it contains a lot of information on your tasks (e.g. regarding the set of certain paramaters or a specific computational trick), and the written mark downs as well as comments contain a lot of information on how things work together as a whole. 

**GENERAL NOTE 2**: 
* Please, when commenting source code, just use English language only. 
* When describing an observation please use English language, too.
* This applies to all exercises throughout this course.

---------------------------------

### <font color="ce33ff">DESCRIPTION</font>:

Before using Shapley values to explain complicated models, it is helpful to understand how they work for simple models.

In this respect the example in this notebook computes a model for the abalone data set (downloaded from UCI) and uses its outputs for explanation of feature importance using the SHAP explainer. In addition, several different visualization techniques (plots) for Shapley values are going to be demonstrated. 

For a description of the features please refer to <a href="https://archive.ics.uci.edu/dataset/1/abalone">UCI abalone data set</a>.

---------------------------------

### <font color="FFC300">TASKS</font>:
The tasks that you need to work on within this notebook are always indicated below as bullet points. 
If a task is more challenging and consists of several steps, this is indicated as well. 
Make sure you have worked down the task list and commented your doings. 
This should be done by using markdown.<br> 
<font color=red>Make sure you don't forget to specify your name and your matriculation number in the notebook.</font>

**YOUR TASKS in this exercise are as follows**:
1. import the notebook to Google Colab or use your local machine.
2. make sure you specified you name and your matriculation number in the header below my name and date. 
    * set the date too and remove mine.
3. read the entire notebook carefully 
    * add comments whereever you feel it necessary for better understanding
    * run the notebook for the first time. 
4. Develop a CNN for image classification and adapt the Shapley Value idea to that model. Comment your entire code.  

## Imports
Import all necessary utilities.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost as xgb
from sklearn.metrics import accuracy_score,confusion_matrix

import shap
shap.initjs()

## Load dataset

In [ ]:
data = pd.read_csv("./data/abalone/abalone.data", names=["sex","length","diameter","height", 
                                                         "whole weight","shucked weight", "viscera weight","shell weight","rings"])
                         
#Get features
y = data['rings']
X = data[["sex","length","height","shucked weight","viscera weight","shell weight"]]

In [ ]:
# do some necessary preprocessing
X.loc[X['sex'] == 'M', 'sex.M'] = 1
X.loc[X['sex'] == 'F', 'sex.F'] = 1
X.loc[X['sex'] == 'I', 'sex.I'] = 1

X = X.drop('sex', axis=1)

### Build a simple ML model and fit it.

In [ ]:
model = xgb.XGBRegressor(objective="reg:squarederror") 
model.fit(X, y)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X)

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred, alpha=0.5, color='b')
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Actual vs. Predicted Values")

# Plot the diagonal line for reference
max_val = max(np.max(y), np.max(y_pred))
min_val = min(np.min(y), np.min(y_pred))
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='dashed')

plt.grid(True)
plt.show()

### <font color=red>NOTE:</font>

Even though we did not put much love into our model, the results are quite ok. However, please keep in mind, that you should start computing Shapley value if and only if you have optimized your model. You do not have to be a genius to understand that the better the model, the better the Shapley values. 

### Now, compute the Shaley values

These are the two essential lines of code. Pass your model into the SHAP Explainer function. 
This creates an explainer object. The, use this to calculate SHAP values for every observation in the feature matrix.

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X)

----------------------------------------------
### Visualization section
----------------------------------------------

#### Waterfall plot for first observation

There are 8 Shapley values for each of the 4177 observations in the feature matrix. 
That is one Shapley value for each feature in your model. You can use the waterfall function to visualise the Shapley values of the first observation.

In [ ]:
shap.plots.waterfall(shap_values[0])

##### <font color="ce33ff">Interpretation:</font>

**E[f(x)] = 9.933** gives the average predicted number of rings across all 4177 abalones. 
**f(x) = 13.043** is the predicted number of rings for this particular abalone. 
The Shapley values are all the values in between. 
For example, the shucked weight contributes to the total predicted number by an increase of **1.68**.

There is a unique waterfall plot for every observation of an abalone in your dataset. They can all be interpreted in the same way as above. In each case, the Shapley values tell us how the features have contributed to the prediction when compared to the average prediction. Large positive/negative values indicate that the feature had a significant impact on the model’s prediction.

#### Forceplot for first observation

In [ ]:
shap.plots.force(shap_values[0])

##### <font color="ce33ff">Interpretation:</font>

Another way to visualise these individual feature contributions is using a so-called force plot. 
Think of it as a condensed waterfall plot. It starts at the same base value of **9.933** and you can see how each feature has contributed to the final prediction of **13.04**.

#### Entire/Stacked force plot

Waterfall and force plots are great for interpreting individual predictions (see above). To understand how your model makes predictions in general you need to aggregate the Shapley values. One way to do this is by using the so-called stacked-force plot.

You can combine multiple force plots together to create a stacked force plot. In this example pass the Shapley values for the first 100 observations in the force plot function. Each individual force plot is now vertical and stacked side by side.

Since this plot is interactive, you can change the ordering of the plots and choose which feature contributions to display.

In [ ]:
shap.plots.force(shap_values[0:100])

##### <font color="ce33ff">Interpretation:</font>

For example, select "shell weight" on x-axis and "shell weight effects" on the y-axis.
Now, from the appearing plot you can see that as shell weight increases the Shapley values also increase. 
Since this is a function of age, older abalones tend to have heavier shells.

This is one way to understand the nature of the relationships captured by the model. 
Now, you will learn that the so-called beeswarm plot and dependence plots can also be used this way.

#### Mean SHAPLEY values

In [ ]:
shap.plots.bar(shap_values)

##### <font color="ce33ff">Interpretation:</font>
This bar plot tells you which features are most important. 
For each and every feature, it calculates the mean Shapley value across all observations. 

Specifically, it takes the mean of the absolute values as it does not want positive and negative values to offset each other. 
There is one bar for each feature in the data set. You can easils see, that "shell weight" has the largest mean Shapley value.

Features that have made large positive/negative contributions will show a large mean Shapley value. In other words, these are the features that have had a significant impact on the model’s predictions. Hence, this bar plot can be used in the same manner as a feature importance plot.

#### Beeswarm plot

The so-called beeswarm plot is one of the most useful plots. The beeswarm visualises all of the SHAP values. 
On the y-axis, the values are grouped by feature. For each group, the colour of the points is determined by the feature value (i.e. higher feature values are more red - see legend ).

In [ ]:
shap.plots.beeswarm(shap_values)

##### <font color="ce33ff">Interpretation:</font>

The features in the above plot are ordered by mean SHAP value.

E.g., for "shell weight", you will notice, that SHAP values increase when the feature value increases. 
Remember, you saw a similar relationship in the stacked force plot. It tells you that larger values for "shell weight" will lead to a higher predicted number of label value "rings".

You may also notice, that the feature "shucked weight" shows the opposite relationship. 
Looking at the beeswarm plot, we can see that larger values for this feature are associated with smaller SHAP values. 

##### Heatmap

Passing the matrix of 200 Shapley values to the heatmap plot function creates a plot with the instances on the x-axis, the model features on the y-axis, and the Shapley values encoded on a color scale. By default, the samples are ordered based on a hierarchical clustering by their explanation similarity. 
This results in samples that have the same model output for the same reason getting grouped together.

In [ ]:
shap.plots.heatmap(shap_values[:200])

##### <font color="ce33ff">Interpretation:</font>

The output of the model is shown above the heatmap matrix (centered around the explaination’s average value), and the global importance of each feature shown as a bar plot on the right hand side of the plot (by default this is the average measure of overall importance).

#### Dependence plots of the Shapley values 

A dependence plot is a scatter plot of the Shapley value vs. the feature value for one single feature. 
They are particularly useful if the feature has got a non-linear relationship with the label.

##### Plot 1: shell weight

In [ ]:
shap.plots.scatter(shap_values[:,"shell weight"])

##### <font color="ce33ff">Interpretation:</font>

For example, take the dependence plot for "shell weight" feature. 
Looking at the beeswarm plot we may have assumed that the Shapley values increase linearly with the feature value. 
However, this dependency plot shows you that the relationship is not perfectly linear. That is very useful.

##### Plot 2: shucked weight

In [ ]:
shap.plots.scatter(shap_values[:,"shucked weight"])

##### <font color="ce33ff">Interpretation:</font>
The dependency plot for "shucked weight" (i.e the weight of the abalone meat). 
Using this plot you can see and confirm the relationship you saw in the beeswarm plot already. 
The SHAP values do decrease as shucked weight increases.

##### Plot 3: shell weight & shucked weight

In [ ]:
shap.plots.scatter(shap_values[:,"shell weight"],
                   color=shap_values[:,"shucked weight"])

##### <font color="ce33ff">Interpretation:</font>
This plot can be used to visualise interactions between features. BUT be cautious! 
In your case, the plot is a result of the correlation between the two features.

Intuitively, this relationship seems strange. 

Wouldn’t you expect an older abalone to be larger and having more meat? 

This is, in fact, a result of an interaction between "shell weight" and "shucked weight". 
You could not see it in the dependence plot due to the correlation. 